In [1]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [21]:
doc = """
우크라이나에 지원할 장갑차 50대를 구매하기 위한 자금 550만달러(약 78억원)가 크라우드펀딩을 통해 하루 만에 모금됐다고 영국 일간 가디언이 3일(현지시간) 보도했다.

보도에 따르면 러시아 침공 직후부터 전쟁 자금 모금 활동을 펴온 우크라이나 방송인 세르히 프라튤라가 설립한 재단이 지난 2일 크라우드펀딩 모금 활동을 개시, 하루만에 목표한 550만달러를 모금했다.
버려진 러시아군 전차에서 부품을 뜯는 우크라이나군 병사들
버려진 러시아군 전차에서 부품을 뜯는 우크라이나군 병사들
[AFP 연합뉴스자료사진. 재판매 및 DB 금지]

이 자금은 영국제 병력 수송장갑차인 FV103 스파르탄 50대를 구매해 돈바스 지역의 최전방에 보내기 위한 것이다.

재단 측은 애초 1주일을 모금 기간으로 생각했으나 모금 개시 6시간 만에 목표액의 절반을 채웠으며 이튿날인 3일 점심 무렵에는 더는 모금을 할 필요가 없어졌다.

FV103 스파르탄은 영국군이 1978년부터 사용한 장갑차로, 우크라이나 전쟁에는 이미 영국 정부가 공식 지원한 35대가 투입돼 상당한 성과를 낸 것으로 전해졌다.

2차 세계대전 등 과거에도 군수 자금 확보를 위한 민간 모금 활동이 없었던 것은 아니지만 우크라이나 전쟁에서는 디지털 시대에 걸맞은 온라인 기반의 모금이 눈에 띈다고 가디언은 전했다.

세르히 프라튤라 재단의 관계자는 "모든 것을 온라인으로 보고 심지어 참호에 앉아있는 사람과도 소셜네트워크로 접근할 수 있다"고 말했다.

이 재단은 그동안 튀르키예(터키)제 무인 전투항공기 바이락타르TB2 3대를 구매하기 위한 1천600만달러를 확보한 바 있으며 이밖에 드론, 차량, 의료키트 등을 사기 위한 자금도 모금해왔다.

우크라이나 정부도 이런 기류에 맞춰 크라우드 펀딩 모금 플랫폼인 '유나이티드24'를 출범했으며 영화 '스타워즈'의 배우 마크 해밀턴에 이어 최근 예일대 교수인 티머시 스나이더를 홍보대사로 임명했다.
"""

In [22]:
JVM_PATH = '/Library/Java/JavaVirtualMachines/zulu-11.jdk/Contents/Home/bin/java' #Mac M1 환경에서 돌아가기 위한 자바 환경변수 설정

okt = Okt(jvmpath=JVM_PATH)

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('우크라이나', 'Noun'), ('에', 'Josa'), ('지원', 'Noun'), ('할', 'Verb'), ('장갑차', 'Noun'), ('50', 'Number'), ('대를', 'Verb'), ('구매', 'Noun'), ('하기', 'Verb')]
명사 추출 : 우크라이나 지원 장갑차 구매 위 자금 약 우드 펀딩 통해 하루 모금 영국 일간 가디언 현지 시간 보도 보도 러시아 침공 직후 전쟁 자금 모금 활동 온 우크라이나 방송인 세르 프라 튤 라가 설립 재단 지난 우드 펀딩 모금 활동 개시 하루 목표 모금 버려진 러시아군 전차 부품 우크라이나 병사 버려진 러시아군 전차 부품 우크라이나 병사 연합뉴스 자료 사진 재판매 및 금지 이 자금 국제 병력 수송 장갑차 스파르탄 구매 돈바스 지역 최 전방 위 것 재단 측은 애초 주일 모금 기간 생각 모금 개시 목표 액 절반 이튿날 점심 무렵 더 모금 필요 스파르탄 영국군 사용 장갑차 우크라이나 전쟁 영국 정부 공식 지원 대가 투입 성과 것 차 세계대전 등 과거 군수 자금 확보 위 민간 모금 활동 것 우크라이나 전쟁 디지털 시대 걸 온라인 기반 모금 눈 가디언 전 세르 프라튤 재단 관계자 모든 것 온라인 보고 심지어 참호 사람 소셜 네트워크 접근 수 고 말 이 재단 그동안 튀르키예 터키 제 무인 전투 항공기 바 이락 타르 구매 위 확보 바 이 드론 차량 의료 키트 등 사기 위 자금 모금 우크라이나 정부 기류 우드 펀딩 모금 플랫폼 유나이티드 를 출범 영화 스타워즈 의 배우 마크 해밀턴 최근 예일대 교수 티머시 스나이더 홍보 대사 임명


In [28]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 308
trigram 다섯개만 출력 : ['가디언 세르' '가디언 세르 프라튤' '가디언 현지' '가디언 현지 시간' '개시 목표']


In [29]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [30]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['러시아 침공 직후', '모금 활동 우크라이나', '자금 모금 우크라이나', '모금 버려진 러시아군', '모금 우크라이나 정부']


In [31]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [33]:
#nr_candidates가 키워드 범위의 다양성 조절
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['보도 보도 러시아', '러시아 침공', '모금 활동 우크라이나', '모금 버려진 러시아군', '모금 우크라이나 정부']